In [ ]:
import arcgis
import time
import os
from arcgis.features import FeatureLayerCollection
from arcgis.features import GeoAccessor
from arcgis.gis import GIS
from shapely import Point
from schema import layer_fields
import pandas as pd

from dotenv import load_dotenv

In [ ]:
load_dotenv()

ARCGIS_SERVER = os.getenv("ARCGIS_SERVER")
ARCGIS_USERNAME = os.getenv("ARCGIS_USERNAME")
ARCIGS_PASSWORD = os.getenv("ARCIGS_PASSWORD")

In [ ]:
gis = arcgis.GIS(ARCGIS_SERVER,ARCGIS_USERNAME,ARCIGS_PASSWORD)
gis

GIS @ https://omniswift.maps.arcgis.com version:2025.2

In [ ]:
# --- CREATE EMPTY SERVICE, THEN ADD A POINT LAYER (READY FOR DATA) ---

# Assumes you're already signed in:
# gis = GIS("home")  # or GIS(url, username, password)

# -----------------------------
# 0) Your attribute fields here
# -----------------------------
# We'll keep your existing fields and ensure OBJECTID is present.
def ensure_oid(fields):
    has_oid = any(
        (f.get("name","").upper() == "OBJECTID") and (f.get("type") == "esriFieldTypeOID")
        for f in fields
    )
    if not has_oid:
        fields = [{
            "name":"OBJECTID",
            "type":"esriFieldTypeOID",
            "alias":"OBJECTID",
            "sqlType":"sqlTypeInteger",
            "nullable": False,
            "editable": False
        }] + list(fields)
    return fields

# <<< replace with your real fields >>>
# Example only; keep your existing layer_fields variable if you already have it.
try:
    layer_fields = layer_fields  # use what you had
except NameError:
    layer_fields = [
        {"name":"fullname","type":"esriFieldTypeString","alias":"Full Name",
         "sqlType":"sqlTypeVarchar","length":255,"nullable":True,"editable":True}
    ]
layer_fields = ensure_oid(layer_fields)

# 1) Unique name (avoid collisions)
unique_name = f"SurveyData_{int(time.time())}"
print("Service name:", unique_name)

# 2) (Optional) remove that known ghost item if it exists
try:
    ghost = gis.content.get("vXaMZMir502uvJfN")
    if ghost:
        print("Deleting ghost item:", ghost.title, ghost.id)
        ghost.delete()
except Exception:
    pass

# 3) Create a fresh folder (correct API: create_folder, not folders.create)
folder = gis.content.create_folder(unique_name)
print("Folder created:", folder.get("id"))

# 4) Item properties (do NOT force "type"; API handles it)
item_properties = {
    "title": unique_name,
    "tags": "ESRI, survey",
    "snippet": "Hosted feature service for survey data",
    "description": "Empty service; point layer will be added via add_to_definition."
}

# 5) Create EMPTY service (no layers yet)
extent = {"xmin": -180, "ymin": -90, "xmax": 180, "ymax": 90, "spatialReference": {"wkid": 4326}}
create_params = {
    "name": unique_name,                      # serviceName (critical)
    "serviceDescription": "Survey data store",
    "capabilities": "Create,Query,Update,Delete,Sync",
    "maxRecordCount": 2000,
    "supportedQueryFormats": "JSON",
    "hasStaticData": False,
    "initialExtent": extent,
    "fullExtent": extent,
    "spatialReference": {"wkid": 4326},
    "allowGeometryUpdates": True,
    "tables": [],
    "layers": []                              # EMPTY now
}

svc_item = gis.content.create_service(
    name=unique_name,                          # also set at top level
    service_type="featureService",
    create_params=create_params,
    item_properties=item_properties,
    folder=folder["id"]
)
print("Empty service created:", svc_item.id, svc_item.url)

# 6) Define the POINT layer to add
feature_layer_definition = {
    "id": 0,
    "name": "MeterInstallations",
    "type": "Feature Layer",
    "geometryType": "esriGeometryPoint",
    "objectIdField": "OBJECTID",
    "displayField": "fullname",
    "fields": layer_fields,
    "extent": extent,
    "allowGeometryUpdates": True,
    "hasZ": False,
    "hasM": False,
    "capabilities": "Create,Query,Update,Delete,Sync"
    # Optional but sometimes helpful:
    # "drawingInfo": {"renderer": {"type": "simple", "symbol": {"type":"esriSMS","style":"esriSMSCircle","size":6}}}
}

# 7) Add the layer with a safe retry (handles portal timing hiccups)
flc = FeatureLayerCollection.fromitem(svc_item)
payload = {"layers": [feature_layer_definition]}

for attempt in range(1, 5):
    try:
        time.sleep(1.5 * attempt)  # brief wait helps provisioning
        flc.manager.add_to_definition(payload)
        print(f"add_to_definition: success on attempt {attempt}")
        break
    except Exception as e:
        print(f"add_to_definition attempt {attempt} failed:", e)
        if attempt == 4:
            raise

# 8) Verify the point layer exists (and print its URL)
flc = FeatureLayerCollection.fromitem(svc_item)  # refresh
if getattr(flc, "layers", None):
    for lyr in flc.layers:
        print("Layer:", lyr.properties.name, "| geometry:", lyr.properties.geometryType, "| URL:", lyr.url)
else:
    raise RuntimeError("Layer was not created — check the layer definition or portal logs.")

Service name: SurveyData_1754653834


c:\Users\Admin\AppData\Local\ESRI\conda\envs\myarcgispro\Lib\site-packages\IPython\core\interactiveshell.py:3550: DeprecatedWarning: create_folder is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `gis.content.folders.create` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Folder created: 8d2e4ccc8db347e28c9975a04b11f2a3
Empty service created: 6b6343b0113b425394906939d7ebb72c https://services6.arcgis.com/vXaMZMir502uvJfN/arcgis/rest/services/SurveyData_1754653834/FeatureServer
add_to_definition: success on attempt 1
Layer: MeterInstallations | geometry: esriGeometryPoint | URL: https://services6.arcgis.com/vXaMZMir502uvJfN/arcgis/rest/services/SurveyData_1754653834/FeatureServer/0


In [ ]:
df = pd.read_csv("../data/sample_data.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
flc = FeatureLayerCollection.fromitem(svc_item)
layer = None
for lyr in flc.layers:
    if lyr.properties.name == "MeterInstallations":
        layer = lyr
        break
if layer is None:
    # fallback: take first layer
    layer = flc.layers[0]
print("Target layer:", layer.properties.name, "| geometry:", layer.properties.geometryType)

Target layer: MeterInstallations | geometry: esriGeometryPoint


In [35]:
csv_path = "../data/sample_data.csv"   # change if needed
lon_src   = "d_longitude"
lat_src   = "d_latitude"

In [37]:
# ---- 1) Load & clean ----
df = pd.read_csv(csv_path)

# keep rows with at least one positive coord (your rule), then coerce to float
df = df.query(f'{lon_src} > 0 | {lat_src} > 0').copy()
df["longitude"] = pd.to_numeric(df[lon_src], errors="coerce")
df["latitude"]  = pd.to_numeric(df[lat_src], errors="coerce")

# drop NaNs and out-of-range coords
df = df.dropna(subset=["longitude", "latitude"])
df = df[(df["longitude"] >= -180) & (df["longitude"] <= 180) &
        (df["latitude"]  >=  -90) & (df["latitude"]  <=  90)]

# short-circuit if nothing valid
if df.empty:
    raise ValueError("No valid rows after cleaning coordinates.")

# ---- 2) Build Spatially Enabled DF (WGS84) ----
sdf = GeoAccessor.from_xy(df, x_column="longitude", y_column="latitude", sr=4326)

# ---- 3) Align to layer schema ----
# get allowed fields from target layer
layer_fields = {f["name"] for f in layer.properties.fields}
protected = {"OBJECTID"}
gid = (layer.properties.get("globalIdField") or "").strip()
if gid:
    protected.add(gid)

# only keep attributes that exist on the layer and are not protected
keep_cols = [c for c in sdf.columns if (c in layer_fields) and (c not in protected)]

# you can map CSV names to layer names here if needed, e.g.:
# rename_map = {"full_name": "fullname", "phone_no": "phone"}
# sdf = sdf.rename(columns=rename_map)
# then recompute keep_cols if you renamed:
# keep_cols = [c for c in sdf.columns if (c in layer_fields) and (c not in protected)]

# ---- 4) Append in batches as a FeatureSet ----
def adds_batch(sdf_batch):
    # build featureset: attributes + geometry (SHAPE comes from sdf.spatial)
    cols = keep_cols + ["SHAPE"]
    fs = sdf_batch[cols].spatial.to_featureset()
    res = layer.edit_features(adds=fs)
    return res

batch_size = 1000
n = len(sdf)
print(f"Preparing to add {n} features to layer: {layer.properties.name}")

start = 0
total_added = 0
errors = []

while start < n:
    end = min(start + batch_size, n)
    print(f"Adding features {start}–{end-1} ...")
    result = adds_batch(sdf.iloc[start:end].copy())

    add_results = result.get("addResults", [])
    total_added += sum(1 for r in add_results if r.get("success"))
    batch_errors = [r.get("error") for r in add_results if not r.get("success")]
    if batch_errors:
        errors.extend(batch_errors)
        print(f"  Warnings: {len(batch_errors)} failures in this batch")

    start = end

print(f"Append complete. Successfully added: {total_added}/{n}")
if errors:
    print(f"{len(errors)} adds failed. First few errors:")
    for e in errors[:5]:
        print("  -", e)

# ---- 5) Verify feature count ----
count_after = layer.query(return_count_only=True)
print("Layer feature count now:", count_after)

Preparing to add 11 features to layer: MeterInstallations
Adding features 0–10 ...


c:\Users\Admin\AppData\Local\ESRI\conda\envs\myarcgispro\Lib\site-packages\pandas\core\dtypes\cast.py:1066: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\Admin\AppData\Local\ESRI\conda\envs\myarcgispro\Lib\site-packages\pandas\core\dtypes\cast.py:1091: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\Admin\AppData\Local\ESRI\conda\envs\myarcgispro\Lib\site-packages\pandas\core\dtypes\cast.py:1066: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\Admin\AppData\Local\ESRI\conda\envs\myarcgispro\Lib\site-packages\pandas\core\dtypes\cast.py:1091: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():


Append complete. Successfully added: 11/11
Layer feature count now: 11


In [ ]:
# import pyodbc
# import os

# # If you're using environment variables, uncomment and use this:
# # from dotenv import load_dotenv
# # load_dotenv()

# # DRIVER = os.getenv("DRIVER")
# # SERVER = os.getenv("SERVER")
# # USER = os.getenv("USER")
# # DATABASE = os.getenv("DATABASE")
# # PASSWORD = os.getenv("PASSWORD")

# # # Direct connection string (hardcoded — not recommended for production)
# # conn_string = r"""
# #     DRIVER={ODBC Driver 18 for SQL Server};
# #     SERVER=nimmes-server.database.windows.net;
# #     DATABASE=nimmes-db-20241707;
# #     UID=nimmes_sever_db_root;
# #     PWD=:H&]S?)8De8'\''ee_G;
# #     Encrypt=yes;
# #     TrustServerCertificate=no;
# #     Authentication=SqlPassword;
# # """

# # try:
# #     conn = pyodbc.connect(conn_string)
# #     print("Connected to the database successfully!")
# # except Exception as e:
# #     print("Error connecting to the database:", e)

# SQL_CONNECTION_STRING="Driver={ODBC Driver 18 for SQL Server};Server=nimmes-server.database.windows.net;Database=nimmes-db-20241707;\
#                                  UID=nimmes_sever_db_root;PWD=:H&]S?)8De8'\''ee_G;Encrypt=yes;TrustServerCertificate=no;Authentication=ActiveDirectoryInteractive"

# conn = pyodbc.connect(SQL_CONNECTION_STRING)